In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader,random_split

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import import_ipynb
from tqdm import tqdm
import warnings

from drive.MyDrive.TranformerNMT.dataset import BiDataset,causal_mask
from drive.MyDrive.TranformerNMT.model import build_transformer
from drive.MyDrive.TranformerNMT.config import get_weights_file_path, get_config

from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!pip install datasets
#!pip install import_ipynb

In [ ]:
from pathlib import Path


def get_or_buid_tokenizer(config, ds, lang):
  tokenizer_path = Path(config['tokenizer_file'].format(lang))
  if not Path.exists(tokenizer_path):
    tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(special_tokens = ["[UNK]","[PAD]","[SOS]","[EOS]"],min_frequency = 2)
    tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer = trainer)
    tokenizer.save(str(tokenizer_path))
  else:
    tokenizer = Tokenizer.from_file(str(tokenizer_path))
  return tokenizer
def get_all_sentences(ds, lang):
  for item in ds:
    yield item['translation'][lang]

In [ ]:
def generate_text(model, source_input, source_mask, tokenizer_tgt, max_len, device):
  sos_id = tokenizer_tgt.token_to_id('[SOS]')
  eos_id = tokenizer_tgt.token_to_id('[EOS]')

  encoder_output = model.encode(source_input, source_mask)

  text = torch.empty(1,1).fill_(sos_id).type_as(source_input).to(device)
  while True:
    if text.size(1) == max_len:
      break
    decoder_mask = causal_mask(text.size(1)).type_as(source_input).to(device)
    decoder_output = model.decode(encoder_output, source_mask, text, decoder_mask)

    project = model.project(decoder_output[:, -1])
    _, next_word = torch.max(project, dim = 1)
    text = torch.cat([text, torch.empty(1,1).fill_(next_word.item()).type_as(source_input).to(device)], dim =1)
    if next_word == eos_id:
      break
  return text.squeeze(0)


In [ ]:
def run_validation(model, val_dataloader, tokenizer_tgt, max_len, device, print_msg):
  model.eval()

  source_text = []
  target_text = []
  predict_text = []


  try:
       # get the console window width
       with os.popen('stty size', 'r') as console:
           _, console_width = console.read().split()
           console_width = int(console_width)
  except:
        # If we can't get the console width, use 80 as default
      console_width = 80

  with torch.no_grad():
    for batch in val_dataloader:

      encoder_input = batch['encoder_input'].to(device)
      encoder_mask = batch['encoder_mask'].to(device)

      model_out = generate_text(model, encoder_input, encoder_mask, tokenizer_tgt, max_len, device)
      model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())
      src_text = batch['src_text'][0]
      tgt_text = batch['tgt_text'][0]


      predict_text.append(model_out_text)
      target_text.append(tgt_text)
      source_text.append(src_text)


      print_msg('-'*console_width)
      print_msg(f"{f'SOURCE: ':>12}{src_text}")
      print_msg(f"{f'TARGET: ':>12}{tgt_text}")
      print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

      break






In [ ]:
def get_ds(config):
  ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}',split = "train")

  tokenizer_src = get_or_buid_tokenizer(config, ds_raw, config["lang_src"])

  tokenizer_tgt = get_or_buid_tokenizer(config, ds_raw, config["lang_tgt"])


  train_size = int(len(ds_raw) * 0.9)
  val_size = len(ds_raw) - train_size

  train_ds_raw , val_ds_raw = random_split(ds_raw ,[train_size, val_size])

  train_ds = BiDataset(train_ds_raw, tokenizer_src,tokenizer_tgt,config["lang_src"],config["lang_tgt"],config["seq_len"])
  val_ds = BiDataset(val_ds_raw, tokenizer_src,tokenizer_tgt,config["lang_src"],config["lang_tgt"],config["seq_len"])

  max_len_src = 0
  max_len_tgt = 0

  for item in ds_raw:
    src_ids = tokenizer_src.encode(item["translation"][config["lang_src"]]).ids
    tgt_ids = tokenizer_tgt.encode(item["translation"][config["lang_tgt"]]).ids

    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))
  train_dataloader = DataLoader(train_ds,batch_size = config["batch_size"], shuffle = True)
  val_dataloader = DataLoader(val_ds,batch_size = 1, shuffle= True)
  return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

In [ ]:
def get_model(config, src_vocab_size, tgt_vocab_size):
  model = build_transformer(src_vocab_size , tgt_vocab_size,config["seq_len"], config["seq_len"],config["d_model"])
  return model

In [ ]:
def train_model(config):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)
  train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
  model = get_model(config,tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

  #writer = SummaryWriter(config["experiment_name"])


  optimizer = torch.optim.Adam(model.parameters(), lr = config["lr"], eps = 1e-9)
  init_epoch = 0
  global_step = 0
  if config["pre_load"]:
    model_filename = get_weights_file_path(config,config["preload"])
    state = torch.load(model_filename)
    init_epoch = state['epoch'] +1
    optimizer.load_state_dict(state["optimizer_state_dict"])
    global_step = state["global_step"]
  loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1).to(device)
  for epoch in range(init_epoch, config["num_epochs"]):
    torch.cuda.empty_cache()
    model.train()
    batch_iter = tqdm(train_dataloader, desc = f'Huan luyen lan thu {epoch:02d}')
    for batch in batch_iter:
      encoder_input = batch["encoder_input"].long().to(device)
      decoder_input = batch["decoder_input"].long().to(device)
      encoder_mask = batch["encoder_mask"].to(device)
      decoder_mask = batch["decoder_mask"].to(device)

      #print(encoder_input.shape)

      encoder_output = model.encode(encoder_input, encoder_mask)
      print(encoder_output.shape)
      decoder_output = model.decode(encoder_output,encoder_mask, decoder_input, decoder_mask)

      projection_output = model.project(decoder_output)

      label = batch["label"].to(device)
      loss = loss_fn(projection_output.view(-1, tokenizer_tgt.get_vocab_size()),label.view(-1))
      batch_iter.set_postfix({f"loss" : f"{loss.item(): 6.3f}"})

      #writer.add_scalar("loi", loss.item(), global_step)
      #writer.flush()

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      global_step +=1

    run_validation(model, val_dataloader, tokenizer_tgt, config['seq_len'], device, lambda text :batch_iter.write(text))
    model_filename = get_weights_file_path(config, f'{epoch:02d}')
    torch.save({

          'epoch':epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          "global_step" : global_step
      },
      model_filename
      )

if __name__ == '__main__':
  warnings.filterwarnings("ignore")

  config = get_config()

  train_model(config)



Huan luyen lan thu 00:   0%|          | 1/3638 [00:00<11:12,  5.41it/s, loss=10.023]

torch.Size([8, 350, 512])


Huan luyen lan thu 00:   0%|          | 2/3638 [00:00<11:47,  5.14it/s, loss=9.918]

torch.Size([8, 350, 512])


Huan luyen lan thu 00:   0%|          | 3/3638 [00:00<12:42,  4.77it/s, loss=9.825]

...

Huan luyen lan thu 02:  97%|█████████▋| 3515/3638 [13:57<00:29,  4.11it/s, loss=6.712]

torch.Size([8, 350, 512])


Huan luyen lan thu 02:  97%|█████████▋| 3516/3638 [13:58<00:28,  4.24it/s, loss=7.103]

torch.Size([8, 350, 512])


Huan luyen lan thu 02:  97%|█████████▋| 3516/3638 [13:58<00:28,  4.24it/s, loss=6.827]

In [ ]:
#config = get_config()
#ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}',split = "train")

#tokenizer_src = get_or_buid_tokenizer(config, ds_raw, config["lang_src"])

#tokenizer_tgt = get_or_buid_tokenizer(config, ds_raw, config["lang_tgt"])
#for item in ds_raw:
 # print(len(item["translation"][config["lang_src"]]))
  #break
#src_ids = tokenizer_src.encode(ds_raw[10]["translation"][config["lang_src"]])
#tgt_ids = tokenizer_tgt.encode(ds_raw[1]["translation"][config["lang_tgt"]]).ids

In [ ]:
#src_ids.tokens
#ds_raw[10]["translation"][config["lang_src"]]

#text = 'hello i am a student'
#text_ids = tokenizer_src.encode(text).ids
#text_ids
